In [14]:
![ -d ../bin/chromedriver-linux64 ] \
    || wget -qq -O ../bin/chromedriver-linux64.zip https://storage.googleapis.com/chrome-for-testing-public/126.0.6478.61/linux64/chromedriver-linux64.zip \
    && unzip -q ../bin/chromedriver-linux64.zip -d ../bin \
    && rm ../bin/chromedriver-linux64.zip

--2024-06-17 10:56:54--  https://storage.googleapis.com/chrome-for-testing-public/126.0.6478.61/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.206.91, 142.250.186.123, 142.250.186.59, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.206.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8771919 (8.4M) [application/zip]
Saving to: ‘../bin/chromedriver-linux64.zip’

../bin/chromedriver 100%[===================>]   8.37M  32.8MB/s    in 0.3s    

2024-06-17 10:56:54 (32.8 MB/s) - ‘../bin/chromedriver-linux64.zip’ saved [8771919/8771919]



In [20]:
![ -d ../bin/chrome-headless-shell-linux64 ] \
    || wget -qq -O ../bin/chrome-headless-shell-linux64.zip https://storage.googleapis.com/chrome-for-testing-public/126.0.6478.61/linux64/chrome-headless-shell-linux64.zip \
    && unzip -q ../bin/chrome-headless-shell-linux64.zip -d ../bin \
    && rm ../bin/chrome-headless-shell-linux64.zip

In [117]:
import bs4
import numpy as np
import pandas as pd
import time

from selenium import webdriver

In [118]:
options = webdriver.ChromeOptions()
options.binary_location = "../bin/chrome-headless-shell-linux64/chrome-headless-shell"
options.add_argument("--no-sandbox")
service = webdriver.ChromeService(executable_path="../bin/chromedriver-linux64/chromedriver")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.flashscore.com/team/poland/2HzmcynI/");
time.sleep(1) # Let JS load

In [119]:
soup = bs4.BeautifulSoup(driver.page_source)

In [120]:
driver.quit()

In [121]:
matches = soup.find_all(class_="event__match")

In [122]:
match = matches[0]
match

<div class="event__match event__match--withRowLink event__match--static event__match--last event__match--twoLine" id="g_1_vwtfSsT6"><a aria-describedby="g_1_vwtfSsT6" class="eventRowLink" href="https://www.flashscore.com/match/vwtfSsT6/#/match-summary" rel="nofollow" target="_blank" title="Click for match detail!"></a><span class="event__check--hidden"></span><div class="event__time">16.06. 13:00</div><div class="_participant_x6lwl_4 event__homeParticipant" data-testid="wcl-matchRow-participant"><img alt="Poland" class="_assetContainer_1nfae_4 _logo_10fi9_4 _sizeXXS_1nfae_18" data-testid="wcl-assetContainerBoxFree-XXS" loading="eager" src="https://static.flashscore.com/res/image/data/hKm3dvB6-vZG58BBc.png"/><span class="_simpleText_zfz11_4 _webTypeSimpleText01_zfz11_8 _name_x6lwl_17" data-testid="wcl-scores-simpleText1">Poland</span></div><div class="_participant_x6lwl_4 event__awayParticipant" data-testid="wcl-matchRow-participant"><img alt="Netherlands" class="_assetContainer_1nfae_4

In [136]:
def parse_match(match: bs4.BeautifulSoup) -> tuple[np.datetime64, str, str, str]:
    # If penalties, remove suffix
    datetime = match.find(class_="event__time").text.lower().removesuffix("pen").split()
    # Check if date-only
    if len(datetime) == 1:
        datetime = pd.to_datetime(datetime[0], dayfirst=True)
    else:
        date, time = datetime
        date = date.split('.')
        # Check if year is present
        if not date[2]:
            date[2] = str(dt.date.today().year)
        datetime = pd.to_datetime('.'.join(date) + ' ' + time, dayfirst=True)

    home_participant = match.find(class_="event__homeParticipant")
    home_participant = (home_participant.find("strong") or home_participant.find("span")).text.lower().strip()
    away_participant = match.find(class_="event__awayParticipant")
    away_participant = (away_participant.find("strong") or away_participant.find("span")).text.lower().strip()

    href = match.find("a").get("href")
    url = urllib.parse.urlparse(href)
    url = url._replace(netloc="m.flashscore.com", fragment='')
    new_href = url.geturl()
    
    return datetime, home_participant, away_participant, new_href

In [137]:
parse_match(matches[3])

(Timestamp('2024-03-26 19:45:00'),
 'wales',
 'poland',
 'https://m.flashscore.com/match/bTFTAeY8/')